1.	Design a custom 4-layer feed forward neural network for the weather prediction dataset. Modify your model to include L2 regularization on all hidden layers.

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F

df = pd.read_csv("weatherAUS.csv", on_bad_lines='skip')

df = df.dropna(subset=["RainTomorrow"])

num_cols = df.select_dtypes(include=["float64", "int64"]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

cat_cols = df.select_dtypes(include=["object"]).columns
df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])

le_dict = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le

X = df.drop("RainTomorrow", axis=1)
y = df["RainTomorrow"]

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

class WeatherPredictor(nn.Module):
    def __init__(self, input_size):
        super(WeatherPredictor, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

model = WeatherPredictor(input_size=X_train.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # L2 regularization

epochs = 20
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

model.eval()
with torch.no_grad():
    preds = model(X_test)
    preds_class = (preds > 0.5).float()
    acc = accuracy_score(y_test, preds_class)
    print(f"\nTest Accuracy: {acc:.4f}")
    print("\nClassification Report:\n", classification_report(y_test, preds_class))

Epoch 1/20, Loss: 0.7105
Epoch 2/20, Loss: 0.7018
Epoch 3/20, Loss: 0.6926
Epoch 4/20, Loss: 0.6830
Epoch 5/20, Loss: 0.6730
Epoch 6/20, Loss: 0.6629
Epoch 7/20, Loss: 0.6525
Epoch 8/20, Loss: 0.6419
Epoch 9/20, Loss: 0.6308
Epoch 10/20, Loss: 0.6192
Epoch 11/20, Loss: 0.6071
Epoch 12/20, Loss: 0.5944
Epoch 13/20, Loss: 0.5811
Epoch 14/20, Loss: 0.5673
Epoch 15/20, Loss: 0.5531
Epoch 16/20, Loss: 0.5387
Epoch 17/20, Loss: 0.5241
Epoch 18/20, Loss: 0.5096
Epoch 19/20, Loss: 0.4954
Epoch 20/20, Loss: 0.4817

Test Accuracy: 0.7860

Classification Report:
               precision    recall  f1-score   support

         0.0       0.78      1.00      0.88     10588
         1.0       1.00      0.04      0.08      3035

    accuracy                           0.79     13623
   macro avg       0.89      0.52      0.48     13623
weighted avg       0.83      0.79      0.70     13623

